## U shape PHP dynamics with multiple liquid slugs - with heat transfer

## wall heat equation

## $\frac{\partial \theta }{\partial \mathcal{T}} = \nabla^2 \theta + W_{evaporator} + H_{condenser}(\theta_{condenser} - \theta_{wall}) + H_{fluid}(\theta_{fluid} - \theta_{wall})$

## liquid heat equation

## $\frac{\partial \theta }{\partial \mathcal{T}} = \nabla^2 \theta + H_{fluid}(\theta_{wall} - \theta_{fluid})$

## dynamics equations
We solve a set of governing equations:

1. Momentum equation:
 ## $\frac{\mathrm{d}^{2} X_{p}}{\mathrm{d} \tau^{2}}+32 \frac{\mathrm{d} X_{p}}{\mathrm{d} \tau}+\omega_{0}^{2} X_{p}=\wp\left(P_{1}-P_{2}\right)$
 
2. Perfect gas law combined with first law of thermodynamics:

 ## $M_{i}=P_{i}^{1 / \gamma}L_{i}$
 
 ## $\theta_{i}=P_{i}^{(\gamma-1) / \gamma}$
 


3. The phase change relations between the mass of vapor and the heat transfer from the wall
 ## $\frac{\mathrm{d} M_{i}}{\mathrm{d} \mathcal{T}}=\int_{X_{start}}^{X_{end}} H_{vapor} (\theta_{wall} - \theta_{vapor}) dX$

 


## nucleate boiling

$nucleatenum = 2$ is the number of nucleation sites. It is distributed randomly for every start time.

$\Delta t = 0.1$ is the time interval for nucleations, for now it is uniform for all nucleation sites, can be modified later on.

$Δθthreshold = 0.01$ is the super heat threshold.

So it means that at time $\Delta t$,  $2\Delta t$,  $3\Delta t$ ... and so on. It will check if the nucleation sites are in the liquid slug region and if the superheat is fulfilled. if so, it will reslease one or several $L=0.02, P=1$ new vapor bubbles

In [1]:
using Revise
using LinearAlgebra
using DifferentialEquations
using Plots
pyplot()

┌ Warning: no Manifest.toml file found, static paths used
└ @ Revise C:\Users\liyux\.julia\packages\Revise\1boD5\src\packagedef.jl:1369


Plots.PyPlotBackend()

In [2]:
cd(dirname(pwd()))
cd("src") 

includet("OneDOHP.jl")
using ..OneDOHP

## Parameters of the PHP

In [3]:
    ω0 = [sqrt(1.2e004)];
    ℘ = [1.2e005];

    θinitial=1.0
    θ  = 0.15;
    θe = 1 + θ;
    θc = 1 - θ;

    
    He  = 3000.0; # H for wall and vapor
    Hc  = 3000.0; # depreciated
    Hwc = 3000.0; # H for wall and condenser liquid
    Hₗ   =  3000.0; # liquid wall heat transfer coeffcient

    
    X0 = 0.5;

    γ = 1.4;

In [4]:
    
    d = 1/30 # dimensionless diameter
    L = 4.0  # total length of the pipe when streched to a 1D pipe
    L2D = 2.0 # the actual length of the bended pipe in the real world
    angle = pi/2 # inclination angle 
    closedornot = true

true

In [5]:
    nucleatenum = 1
    Xstations = sort(rand(nucleatenum).*L)
    Δt = 0.05

0.05

In [6]:
Xstations

1-element Array{Float64,1}:
 1.2581181303000593

## Pipe definition

In [7]:
tube = Tube(d,L,L2D,angle,closedornot);

## Evaporator and condenser definition

In [8]:
# Xe = map(tuple, [0.0], [2.0])
Xe = map(tuple, [1.0], [3.0])
# Xe = map(tuple, [0.0,3.0], [1.0,4.0])
We = [200.0]
evap = Evaporator(He,θe,Xe,We);

In [9]:
# Xc = map(tuple, [2.0], [4.0])
# Xc = map(tuple, [1.0], [3.0])
Xc = map(tuple, [0.0,3.0], [1.0,4.0])
cond = Condenser(Hc,θc,Xc,Hwc);

## Vapor plug and liquid slug initialization

In [10]:
N=3000
X0 = [(1.0, 3.0)]
dXdt0 = [(0.0,0.0)]
ρₗ = 1000
Xarrays,θarrays = constructXarrays(X0,N,θinitial,L)
liquids=Liquid(γ,Hₗ,ρₗ,ω0,℘,X0,dXdt0,Xarrays,θarrays);

In [11]:
# P = [1.0,1.0,1.0,1.0,1.0]; # closed end
P = [1.0];
δ = 0.05*P;
vapors=Vapor(γ,P,δ);

In [12]:
α = 1 # nondimensional thermal diffusivity = (αₐ d^2) / (νₗ height^2)
Xwallarray,θwallarray = constructXarrays(L,N,θinitial)

wall = Wall(α,Δt,Xstations,Xwallarray,θwallarray);

In [13]:
walltoliquid, liquidtowall = constructmapping(Xarrays,Xwallarray,closedornot,L)
mapping = Mapping(walltoliquid,liquidtowall);

## Construct the system

In [14]:
sys0 = PHPSystem(tube,evap,cond,liquids,vapors,wall,mapping);

In [15]:
typeof(sys0)

PHPSystem

## Construct U for wall heat function

## construct state vector U

In [16]:
δ

1-element Array{Float64,1}:
 0.05

In [17]:
Lvaporplug = XptoLvaporplug(X0,sys0.tube.L,sys0.tube.closedornot)
M = P.^(1/γ).* Lvaporplug


u=[XMδtovec(X0,dXdt0,M,δ); wallθtovec(sys0.wall.θarray); liquidθtovec(sys0.liquid.θarrays)];

In [18]:
# plot([sys0.wall.Xarray],[sys0.wall.θarray], title="initial temperature condition", labels = "θ", xlabel="X", ylabel="θ")

## Solve the constructed ODE for U

In [19]:
cb_boiling =  DiscreteCallback(boiling_condition,boiling_affect!)
cb_merging =  DiscreteCallback(merging_condition,merging_affect!)

cb = CallbackSet(cb_boiling,cb_merging)

CallbackSet{Tuple{},Tuple{DiscreteCallback{typeof(boiling_condition),typeof(boiling_affect!),typeof(DiffEqBase.INITIALIZE_DEFAULT),typeof(DiffEqBase.FINALIZE_DEFAULT)},DiscreteCallback{typeof(merging_condition),typeof(merging_affect!),typeof(DiffEqBase.INITIALIZE_DEFAULT),typeof(DiffEqBase.FINALIZE_DEFAULT)}}}((), (DiscreteCallback{typeof(boiling_condition),typeof(boiling_affect!),typeof(DiffEqBase.INITIALIZE_DEFAULT),typeof(DiffEqBase.FINALIZE_DEFAULT)}(boiling_condition, boiling_affect!, DiffEqBase.INITIALIZE_DEFAULT, DiffEqBase.FINALIZE_DEFAULT, Bool[1, 1]), DiscreteCallback{typeof(merging_condition),typeof(merging_affect!),typeof(DiffEqBase.INITIALIZE_DEFAULT),typeof(DiffEqBase.FINALIZE_DEFAULT)}(merging_condition, merging_affect!, DiffEqBase.INITIALIZE_DEFAULT, DiffEqBase.FINALIZE_DEFAULT, Bool[1, 1])))

In [ ]:
tstep=1e-4

p = sys0
tspan = (0.0, 0.2);
prob = ODEProblem(ODE_innertube, u, tspan, p)

integrator = init(prob, RK4(), dt=tstep, callback=cb);


solve!(integrator)



In [ ]:
p = deepcopy(getcurrentsys(integrator.sol[end],sys0));
p.liquid.Xp

In [ ]:
sol = integrator.sol
sysfinal = getcurrentsys.(sol.u,[sys0]);

In [ ]:
# closed loop plot
@gif for i=1:500:length(sysfinal)

    Xpvapor = getXpvapor(sysfinal[i].liquid.Xp,sysfinal[i].tube.L,sysfinal[i].tube.closedornot)
    θvapor  = (sysfinal[i].vapor.P).^((γ-1)/γ)
    Xp = sysfinal[i].liquid.Xp
    
    p = plot([sysfinal[i].wall.Xarray],[sysfinal[i].wall.θarray], linewidth=3, color = [:black], labels = "θwall", xlabel="X", ylabel="θ", title=("dimensionless temperatures"),xlim=(0,4), ylim=(0.8,1.2))
    
    
    j=1
    while j <= length(Xp)
        if Xp[j][end] >= Xp[j][1]
            plot!(p, [sysfinal[i].liquid.Xarrays[j]],[sysfinal[i].liquid.θarrays[j]], linewidth=2, labels = string("θliquid ",j))
            else 
            # find the index at the end
            index = findfirst(x->x <= sysfinal[i].liquid.Xarrays[j][end], sysfinal[i].liquid.Xarrays[j])
            
            plot!(p, [sysfinal[i].liquid.Xarrays[j][1:index-1]],[sysfinal[i].liquid.θarrays[j][1:index-1]], linewidth=2, labels = string("θliquid ",j))
            plot!(p, [sysfinal[i].liquid.Xarrays[j][index:end]],[sysfinal[i].liquid.θarrays[j][index:end]], linewidth=2, labels = string("θliquid ",j))

        end
        
        j += 1
    end

        j=1
    while j <= length(Xpvapor)
        if Xpvapor[j][end] >= Xpvapor[j][1]
            plot!(p,[Xpvapor[j][1],Xpvapor[j][end]],[θvapor[j]], labels = string("θvapor ",j))
            else
            
            plot!(p,[0.0,Xpvapor[j][end]],[θvapor[j]], labels = string("θvapor ",j))
            plot!(p,[Xpvapor[j][1],L],[θvapor[j]], labels = string("θvapor ",j))

        end
        
        j += 1
    end
    
#     j = 1
#     plot!(p,[0.0,Xpvapor[j][end]],[θvapor[j]], labels = string("θvapor ",j))
#     plot!(p,[Xpvapor[j][1],L],[θvapor[j]], labels = string("θvapor ",j))
    
#     j = 2
#     while j  <= length(θvapor) 
    
#         plot!(p,[Xpvapor[j][1],Xpvapor[j][end]],[θvapor[j]], labels = string("θvapor ",j))
        
#         j += 1
#     end

    vline!( p, [collect(0:2:4)], color=[:blue,:red], width=196, alpha=[0.3,0.3,0.3],labels = "")
    
    end 

In [ ]:
constant = 2.5e6/461.5/300

In [ ]:
function onePtooneT(P,constant)
    1/(1-(log(P)/constant))
end

In [ ]:
onePtooneT(1.0,constant)

In [ ]:
XX = constructXarrays(L::Float64,N,θinitial)

XX[1][1]

In [ ]:
plot(sys_to_heatflux(sysfinal[end]))

In [ ]:
@time ODE_innertube(u,sys0,0.0)

In [ ]:
@time ODE_innertube(u,sys0,0.0);

In [ ]:
@time index_dynamics_end = findfirst(x->abs(x+1e10) <= 10^(-1), u)

In [ ]:
@time newsys = getcurrentsys(u,sys0)

In [ ]:
cd(dirname(pwd()))
cd("src") 

includet("OneDOHP.jl")
using ..OneDOHP